In [1]:
import os
import sys
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
print("Done importing..!")

Done importing..!


In [2]:
files = os.listdir(".")
print(files)

['Iris.csv', 'Untitled3.ipynb', '.ipynb_checkpoints', 'Untitled.ipynb', 'iris-species.zip', 'Untitled1.ipynb', 'Untitled2.ipynb', 'database.sqlite', 'Iris.ipynb']


In [3]:
df = pd.read_csv("Iris.csv")
# print(df.head(5))
print("=="*50)
# print(df.describe())
# print(df['Species'].unique())
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
Id               150 non-null int64
SepalLengthCm    150 non-null float64
SepalWidthCm     150 non-null float64
PetalLengthCm    150 non-null float64
PetalWidthCm     150 non-null float64
Species          150 non-null object
dtypes: float64(4), int64(1), object(1)
memory usage: 7.1+ KB
None


In [4]:
xTrain, xTest, yTrain, yTest = train_test_split(df.drop('Species',axis=1), df['Species'], test_size = 0.2, random_state = 5)
print("Train Length: {} Test Length: {}".format(len(xTrain),len(xTest)))

Train Length: 120 Test Length: 30


In [5]:
le = preprocessing.LabelEncoder()
dum = pd.get_dummies(yTrain)

In [55]:
trainFeatures = xTrain.values
testFeatures = xTest.values
trainLabel = pd.get_dummies(yTrain).values
testLabel = pd.get_dummies(yTest).values

In [56]:
# trainFeatures = StandardScaler().fit_transform(trainFeatures) # stabilize data
# testFeatures = StandardScaler().fit_transform(testFeatures) # stabilize data

In [57]:
softmax = lambda x: np.exp(x) / np.exp(x).sum(axis=1, keepdims=True)
sigmoid = lambda x : 1 / (1+np.exp(-x))  # Replace 0 with your sigmoid calculation.
sigmoid_derivative = lambda x: (1-sigmoid(x))
loss_fn = lambda yhat, y:  -np.sum(y * np.log(yhat))

In [58]:
class NeuralNetwork(object):
    def __init__(self, input_nodes, hidden_nodes, output_nodes, learning_rate):
        # Set number of nodes in input, hidden and output layers.
        self.input_nodes = input_nodes
        self.hidden_nodes = hidden_nodes
        self.output_nodes = output_nodes
        self.lr = learning_rate
        self.init_weight()
        
        
    def init_weight(self):
        self.weight_ih = np.random.normal(0.0, self.input_nodes**-0.5, (self.input_nodes, self.hidden_nodes))
        self.weight_ho = np.random.normal(0.0, self.hidden_nodes**-0.5, (self.hidden_nodes, self.output_nodes))
        

    def zero_grad(self):
        self.dw_ih =  np.zeros(self.weight_ih.shape)
        self.dw_ho = np.zeros(self.weight_ho.shape)
        

    def train(self, features, targets):
        n_records = features.shape[0]
        
        self.zero_grad()
        final_outputs, hidden_outputs = self.forward(features)
        self.backward(final_outputs, hidden_outputs, features, targets)
        self.update_weights(n_records)
        
        loss = loss_fn(final_outputs, targets)
        return loss
    
    
    def validate(self, features, targets):
        final_outputs, hidden_outputs = self.forward(features)
        loss = loss_fn(final_outputs, targets)
        return loss

        
    def forward(self, X):
        #### Implement the forward pass here ####
        hidden_inputs = np.dot(X, self.weight_ih) # signals into hidden layer
        hidden_outputs = sigmoid(hidden_inputs) # signals from hidden layer
        
        final_inputs = np.dot(hidden_outputs, self.weight_ho) # signals into final output layer
        final_outputs = softmax(final_inputs) # signals from final output layer

        return final_outputs, hidden_outputs


    def backward(self, final_outputs, hidden_outputs, X, y):
        # TODO: Output error - Replace this value with your calculations.
        error = final_outputs - y # Output layer error is the difference between desired target and actual output.
        error_term = np.dot(hidden_outputs.T, error)
        
        hidden_error = np.dot(error,  self.weight_ho.T) * hidden_outputs * (1 - hidden_outputs)
        hidden_error_term = np.dot(X.T, hidden_error)
        
        
        # Weight step (input to hidden)
        self.dw_ih += hidden_error_term
        self.dw_ho += error_term


    def update_weights(self, n_records):
        self.weight_ho -= self.lr * self.dw_ho / n_records # update hidden-to-output weights with gradient descent step
        self.weight_ih -= self.lr * self.dw_ih / n_records # update input-to-hidden weights with gradient descent step
    

    def run(self, features):
        #### Implement the forward pass here ####
        final, hidden = self.forward(features)
        return final

In [59]:
input_node = trainFeatures.shape[1]
hidden_node = 32
output_node = trainLabel.shape[1]
lr = 0.3
network = NeuralNetwork(input_node, hidden_node, output_node, lr)


In [60]:

iterations = 10000
for epoch in range(iterations):
#     batch_train = np.random.choice(len(trainFeatures), size=120)
#     X_train, y_train = trainFeatures[batch_train], trainLabel[batch_train]
    
    train_loss = network.train(trainFeatures, trainLabel)
    valid_loss = network.validate(testFeatures, testLabel)
    
    sys.stdout.write("\rIteration {} Progress: {:2.1f}".format(epoch, 100 * epoch/float(iterations)) \
                     + "% ... Training loss: " + str(train_loss)[:5] \
                     + " ... Validation loss: " + str(valid_loss)[:5])
    sys.stdout.flush()

Iteration 9999 Progress: 100.0% ... Training loss: 54.24 ... Validation loss: 15.28